In [ ]:
import open3d as o3d
import open3d.visualization.gui as gui
import open3d.visualization.rendering as rendering
import threading
import numpy as np
from open3d.web_visualizer import _global_jupyter_loop

In [ ]:
def web_draw(geometry):
    
    def add_window():
        app = gui.Application.instance
        window = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
        window.add_geometry("Torus", geometry)
        window.reset_camera_to_default()
        app.add_window(window)
        return window.uid
        
    uid = _global_jupyter_loop.run_sync(add_window)
    print(f"Newly add Window: {uid}")
    visualizer = o3d.WebVisualizer(window_uid=uid)
    visualizer.show()

In [ ]:
cube_red = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_red.compute_vertex_normals()
cube_red.paint_uniform_color((1.0, 0.0, 0.0))
web_draw(cube_red)

In [ ]:
cube_blue = o3d.geometry.TriangleMesh.create_box(1, 2, 4)
cube_blue.compute_vertex_normals()
cube_blue.paint_uniform_color((0.0, 0.0, 1.0))
web_draw2(cube_blue)

In [ ]:
# def box():
#     app = gui.Application.instance
#
#     torus = o3d.geometry.TriangleMesh.create_box()
#     torus.compute_vertex_normals()
#     mat = rendering.Material()
#     mat.shader = "defaultLit"
#
#     w = o3d.visualization.O3DVisualizer("Open3D", 640, 480)
#     w.add_geometry("box", torus, mat)
#     w.reset_camera_to_default()
#     print("my uid", w.uid)
#     app.add_window(w)